In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install music21

In [ ]:
# All imports
import glob
import pickle
import numpy as np
import datetime
import os
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation
from tensorflow.keras.utils import to_categorical

# --- Configuration ---
# Define the file path to your Google Drive project folder
DRIVE_PATH = "/content/drive/My Drive/AI_Music_Composer/"

# --- Part 1: Data Processing and Training Functions ---

def get_notes():
    """ Get all the notes and chords from the midi files in the Google Drive directory """
    notes = []

    # Updated glob path to point to Google Drive
    for file in glob.glob(DRIVE_PATH + "midi_files/*.mid"):
        try:
            midi = converter.parse(file)
            print(f"Parsing {file}")
            notes_to_parse = None
            parts = instrument.partitionByInstrument(midi)
            if parts:
                notes_to_parse = parts.parts[0].recurse()
            else:
                notes_to_parse = midi.flat.notes
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        except Exception as e:
            print(f"Could not parse {file}. Reason: {e}")

    # Create a 'data' directory in your project folder if it doesn't exist
    if not os.path.exists(DRIVE_PATH + 'data'):
        os.makedirs(DRIVE_PATH + 'data')

    with open(DRIVE_PATH + 'data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    pitchnames = sorted(list(set(item for item in notes)))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input = []
    network_output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
    network_output = to_categorical(network_output, num_classes=n_vocab)
    return (network_input, network_output)

def create_network(network_in, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(512, input_shape=(network_in.shape[1], network_in.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

def train_network():
    """ Train a Neural Network to generate music """
    print("--- Starting Data Processing ---")
    notes = get_notes()
    n_vocab = len(set(notes))
    print("--- Preparing Sequences for Training ---")
    network_in, network_out = prepare_sequences(notes, n_vocab)
    print("--- Creating Neural Network ---")
    model = create_network(network_in, n_vocab)
    print("--- Starting Model Training ---")
    model.fit(network_in, network_out, epochs=100, batch_size=64)
    model.save(DRIVE_PATH + 'music_model.h5')
    print("--- Training Complete and Model Saved ---")

# --- Part 2: Music Generation Functions ---

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network """
    start = np.random.randint(0, len(network_input) - 1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # THE FIX IS HERE: Create a *copy* of the pattern
    pattern = list(network_input[start])
    prediction_output = []

    print("--- Generating 500 notes ---")
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction = model.predict(prediction_input, verbose=0)
        top_indices = prediction[0].argsort()[-5:][::-1]
        probabilities = prediction[0][top_indices]
        probabilities = probabilities / np.sum(probabilities)
        index = np.random.choice(top_indices, p=probabilities)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]
    print("--- Note Generation Complete ---")
    return prediction_output

def create_midi(prediction_output, drive_path):
    """ Convert the output from the prediction to notes and create a midi file """
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_path = f'{drive_path}composition_{timestamp}.mid'

    midi_stream.write('midi', fp=file_path)
    print(f"--- Successfully Created MIDI file: {file_path} ---")

def generate():
    """ Main function to generate a piano midi file """
    print("--- Loading Note Data ---")
    with open(DRIVE_PATH + 'data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)
    pitchnames = sorted(list(set(notes)))
    n_vocab = len(set(notes))

    print("--- Loading Trained Model ---")
    model = load_model(DRIVE_PATH + 'music_model.h5')

    print("--- Preparing Sequences for Generation ---")
    # Note: We only need network_input for the starting patterns
    network_input, _ = prepare_sequences(notes, n_vocab)

    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output, DRIVE_PATH)

In [ ]:
train_network()

/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2001 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(


Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_se5.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_esp4.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_se1.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_esp1.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_se7.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_se3.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_esp5.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_esp2.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_se6.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_esp3.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_se2.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_se8.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/alb_se4.mid
Parsing /content/drive/My Drive/AI_Music_Compo

/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=7, channel=None, data=b'Copyright 2004 by Bernd Kr\xfcger.'>; getting generic Instrument
  warnings.warn(


Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/bach_847.mid


/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Pr\xe4ludium und Fuge in D-Dur, BWV 850'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=8, channel=None, data=b'Copyright 1997 by Bernd Kr\xfcger.'>; getting generic Instrument
  warnings.warn(


...

Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn15_1.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn15_12.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn16_6.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn15_9.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn15_10.mid


/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Tr\xe4umerei'>; getting generic Instrument
  warnings.warn(


Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn15_7.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn16_3.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn16_4.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn15_8.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn16_7.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn15_2.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn15_13.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn15_4.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/scn15_3.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/ty_august.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/ty_september.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/ty_november.mid
Parsing /content/drive/My Drive/AI_Music_Composer/midi_files/ty_februar.mid
Parsing /content/drive/My Drive/AI_M

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 17s 74ms/step - loss: 4.9232
Epoch 2/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 19s 75ms/step - loss: 4.5317
Epoch 3/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 21s 76ms/step - loss: 4.4828
Epoch 4/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 21s 78ms/step - loss: 4.4202
Epoch 5/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 21s 79ms/step - loss: 4.3836
Epoch 6/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 14s 80ms/step - loss: 4.3471
Epoch 7/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 14s 81ms/step - loss: 4.2974
Epoch 8/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 21s 83ms/step - loss: 4.2388
Epoch 9/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 20s 82ms/step - loss: 4.1992
Epoch 10/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - loss: 4.0953
Epoch 11/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 21s 82ms/step - loss: 4.0318
Epoch 12/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 14s 82ms/step - loss: 3.9422
Epoch 13/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - loss: 3.8107
Epoch 14/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 14s 82ms/step - loss: 3.7595
Epoch 15/100
17

In [ ]:
generate()

--- Loading Note Data ---
--- Loading Trained Model ---


--- Preparing Sequences for Generation ---
--- Generating 500 notes ---
--- Note Generation Complete ---
--- Successfully Created MIDI file: /content/drive/My Drive/AI_Music_Composer/composition_2025-08-30_13-38-51.mid ---


In [5]:
!git clone https://vivekrsingh10:<YOUR GITHUB TOKEN>@github.com/vivekrsingh10/AI-Music-Composer.git

Cloning into 'AI-Music-Composer'...


In [6]:
!mkdir -p /content/AI-Music-Composer/data

In [7]:
!cp "/content/drive/My Drive/AI_Music_Composer/music_model.h5" /content/AI-Music-Composer/

In [8]:
!cp "/content/drive/My Drive/AI_Music_Composer/data/notes" /content/AI-Music-Composer/data/

In [10]:
!cp "/content/drive/My Drive/Colab Notebooks/AI_Music_Composer.ipynb" /content/AI-Music-Composer/

In [11]:
!cp "/content/drive/My Drive/AI_Music_Composer/sample_output_1.mid" /content/AI-Music-Composer/

In [12]:
!cp "/content/drive/My Drive/AI_Music_Composer/sample_output_2.mid" /content/AI-Music-Composer/

In [13]:
# The content of your comparison script
comparison_code = """
from music21 import converter
import sys

def compare_files(file1, file2):
    \"\"\"Compares two MIDI files and prints the differences.\"\"\"
    try:
        midi_1 = converter.parse(file1)
        midi_2 = converter.parse(file2)

        notes_1 = [str(element.pitch) if hasattr(element, 'pitch') else '.'.join(str(n) for n in element.normalOrder) for element in midi_1.flat.notes]
        notes_2 = [str(element.pitch) if hasattr(element, 'pitch') else '.'.join(str(n) for n in element.normalOrder) for element in midi_2.flat.notes]

        print(f"--- Comparing {file1} and {file2} ---")
        differences_found = 0
        for i in range(min(len(notes_1), len(notes_2))):
            if notes_1[i] != notes_2[i]:
                differences_found += 1
                print(f"Difference at note #{i+1}:")
                print(f"  - {file1}: {notes_1[i]}")
                print(f"  - {file2}: {notes_2[i]}")
                print("-" * 20)

        if differences_found == 0:
            print("No differences found.")
        else:
            print(f"Found a total of {differences_found} differing notes.")

    except FileNotFoundError as e:
        print(f"Error: Could not find a file. Please check filenames. Details: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    # Example usage: Takes filenames from command line arguments
    # python compare_midi.py file1.mid file2.mid
    if len(sys.argv) != 3:
        print("Usage: python compare_midi.py <file1.mid> <file2.mid>")
    else:
        compare_files(sys.argv[1], sys.argv[2])
"""

# Write the code to a new file named 'compare_midi.py'
with open("/content/AI-Music-Composer/compare_midi.py", "w") as f:
    f.write(comparison_code)

print("Created compare_midi.py successfully.")

Created compare_midi.py successfully.


In [14]:
%cd /content/AI-Music-Composer

/content/AI-Music-Composer


In [15]:
!git config user.email "vksingh10may@gmail.com"
!git config user.name "vivekrsingh10"

In [16]:
!git add .

In [17]:
!git commit -m "Add all project files including model, data, and samples"

[main (root-commit) 17664b9] Add all project files including model, data, and samples
 6 files changed, 41 insertions(+)
 create mode 100644 AI_Music_Composer.ipynb
 create mode 100644 compare_midi.py
 create mode 100644 data/notes
 create mode 100644 music_model.h5
 create mode 100644 sample_output_1.mid
 create mode 100644 sample_output_2.mid


In [18]:
!git push

Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (9/9), 38.49 MiB | 8.46 MiB/s, done.
Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), done.
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:     
remote:       —— GitHub Personal Access Token ——————————————————————
remote:        locations:
remote:          - commi

In [20]:
!cp "/content/drive/My Drive/Colab Notebooks/AI_MUsi.ipynb" /content/AI-Music-Composer/

cp: cannot stat '/content/drive/My Drive/Colab Notebooks/Your_Notebook_Name.ipynb': No such file or directory
